# Synthetic Data Examples

In [ ]:
# Uncomment the following line when running on Google Colab
# !pip install "autora[synthetic-data]" --pre

Load the Weber-Fechner Law:

In [ ]:
import numpy as np
from autora.synthetic.psychophysics.weber_fechner_law import weber_fechner_law
s = weber_fechner_law(random_state=None)

Check the docstring to get information about the model

In [ ]:
help(weber_fechner_law)

... or use the describe function:

In [ ]:
from autora.synthetic.utilities import describe

print(describe(s))

The synthetic experiement `s` has properties like the name of the experiment:

In [ ]:
s.name

... a valid metadata description:

In [ ]:
s.variables

... a function to generate the full domain of the data (if possible)

In [ ]:
x = s.domain()
x

... the experiment_runner runner which can be called to generate experimental results:

In [ ]:
import numpy as np
y = s.experiment_runner(x)  # doctest: +ELLIPSIS
y

... a function to plot the ground truth:

In [ ]:
s.plotter()

... against a fitted model if it exists:

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(x, y)
s.plotter(model)

These can be used to run a full experimental cycle

In [ ]:
from autora.workflow.protocol import ResultKind
from autora.experimentalist.pipeline import make_pipeline
from autora.experimentalist.pooler.grid import grid_pool
from autora.experimentalist.sampler.random_sampler import random_sampler
from functools import partial
import random
variables = s.variables
pool = partial(grid_pool, ivs=variables.independent_variables)
random.seed(181) # set the seed for the random sampler
sampler = partial(random_sampler, n=20)
experimentalist_pipeline = make_pipeline([pool, sampler])

from autora.workflow import Controller
theorist = LinearRegression()

cycle = Controller(
    variables=variables, experimentalist=experimentalist_pipeline,
    experiment_runner=s.experiment_runner, theorist=theorist,
    monitor=lambda s: (s.history[-1].kind == ResultKind.MODEL) and
                       print(f"finished cycle {len(s.models)}"))

c = cycle.run(10)
best_model = c.state.models[-1]
print(f"I = "
      f"{best_model.coef_[0]:.2f} S0 "
      f"{best_model.coef_[1]:+.2f} S1 "
      f"{best_model.intercept_:+.2f}")
